<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: AMES Housing Dataset

--- 
# Notebook 4

The fourth notebook will apply the data changes in notebook 2 and use the ridge regression model trained in notebook 3 to predict the target variable for Kaggle Submission.

---

# 1.0 Data and libraries Import

In [1]:
# Import libraries
import numpy as np
import math
import statistics
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV

In [2]:
# Load data in dataframe
X = pd.read_csv('datasets/test.csv')

# 2.0 Filling missing values according to Notebook 1

In [3]:
X.columns[X.isnull().any()]

Index(['Lot Frontage', 'Alley', 'Mas Vnr Type', 'Mas Vnr Area', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2',
       'Electrical', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
       'Garage Finish', 'Garage Qual', 'Garage Cond', 'Pool QC', 'Fence',
       'Misc Feature'],
      dtype='object')

In [4]:
# Putting all variables into a list and check the datatypes of the missing variables
nan_vars = [
    'Lot Frontage', 'Alley', 'Mas Vnr Type', 'Mas Vnr Area', 'Bsmt Qual', 
    'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 
    'Electrical', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt', 
    'Garage Finish', 'Garage Qual', 'Garage Cond', 'Pool QC', 'Misc Feature']

# Find the number of misssing values in each variable
X[nan_vars].isnull().sum().sort_values(ascending = False)

Pool QC           874
Misc Feature      837
Alley             820
Fireplace Qu      422
Lot Frontage      160
Garage Yr Blt      45
Garage Finish      45
Garage Qual        45
Garage Cond        45
Garage Type        44
Bsmt Cond          25
Bsmt Exposure      25
BsmtFin Type 1     25
BsmtFin Type 2     25
Bsmt Qual          25
Mas Vnr Area        1
Mas Vnr Type        1
Electrical          1
dtype: int64

In [5]:
X[X['Lot Frontage'].isnull()]
X.iloc[764, X.columns.get_loc('Garage Finish')] = 'Unf'
X.iloc[764, X.columns.get_loc('Garage Qual')] = 'TA'
X.iloc[764, X.columns.get_loc('Garage Cond')] = 'TA'
X.iloc[764, X.columns.get_loc('Garage Yr Blt')] = X.iloc[764, X.columns.get_loc('Year Built')]
X['Mas Vnr Type'] = np.where(X['Mas Vnr Type'].isnull(), 'None', X['Mas Vnr Type'])
X['Mas Vnr Area'] = np.where(X['Mas Vnr Area'].isnull(), 0, X['Mas Vnr Area'])
X['Electrical'] = X['Electrical'].fillna('FuseA')

## 3.1 Create Ordinal Data according to Notebook 2

In [6]:
X['Misc Feature'].fillna('None', inplace = True)
X['Garage Yr Blt'] = X['Garage Yr Blt'].fillna(X['Year Built'])
X.fillna(0, inplace = True)
X['Lot Shape'] = X['Lot Shape'].map({'Reg':3,'IR1':2,'IR2':1,'IR3':0})
X['Land Slope'] = X['Land Slope'].map({'Gtl':2,'Mod':1,'Sev':0})
X['Electrical'] = X['Electrical'].map({'SBrkr':4,'FuseA':3,'FuseF':1, 'FuseP':0, 'Mix':2})
X['Functional'] = X['Functional'].map({'Typ':7,'Min1':6,'Min2':5,'Mod':4,'Maj1':3,'Maj2':2,'Sev':1,'Sal':0})
X['Paved Drive'] = X['Paved Drive'].map({'Y':2, 'P':1, 'N':0})
X['Garage Finish'] = X['Garage Finish'].map({'Fin':3,'RFn':2,'Unf':1,'None':0, 0:0})
X['BsmtFin Type 1'] = X['BsmtFin Type 1'].map({'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'None':0, 0:0})
X['BsmtFin Type 2'] = X['BsmtFin Type 2'].map({'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'None':0, 0:0})
X['Bsmt Exposure'] = X['Bsmt Exposure'].map({'Gd':3,'Av':2,'Mn':1,'No':0,'None':0, 0:0})
X['Fence'] = X['Fence'].map({'GdPrv':4,'GdWo':3,'MnPrv':2,'MnWw':1,'None':0, 0:0})
X['Alley'] = X['Alley'].map({'Pave':2,'Grvl':1, 0:0})
rank_list = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':0, 'Po':-2, 0:0}
ordinal_vars = ['Exter Qual', 'Exter Cond' , 'Bsmt Qual', 'Bsmt Cond', 'Heating QC', 'Kitchen Qual', 'Fireplace Qu', 'Garage Qual', 'Garage Cond', 'Pool QC']
for var in ordinal_vars:
    X[var] = X[var].map(rank_list)
rank_list = {'Y':1, 'N':0}
X['Central Air'] = X['Central Air'].map(rank_list)
X['MS SubClass'] = X['MS SubClass'].map({20:15, 30:4, 40:3, 45:1, 50:12, 60:14, 70:7, 75:13, 80:9, 85:5, 90:8, 120:11, 150:2, 160:10, 180:2, 190:6})
X['Bldg Type'] = X['Bldg Type'].map({'1Fam':5, 'TwnhsE':4, 'Duplex':3, 'Twnhs':2, '2fmCon':1})

## 3.2 Dumbified Variables

In [7]:
dumb_vars = ['MS Zoning', 'Street', 'Land Contour', 'Condition 1', 'Condition 2', 
             'House Style', 'Lot Config', 'Neighborhood', 'Roof Style', 'Roof Matl', 
             'Exterior 1st', 'Exterior 2nd', 'Foundation', 'Heating', 'Garage Type', 
             'Mas Vnr Type', 'Misc Feature', 'Sale Type']

for var in dumb_vars:
    X = pd.get_dummies(data=X, columns = [var], dummy_na = True, prefix = var)

## 3.3 Variables to be Drop

In [8]:
X.drop(columns = ['Utilities'], inplace = True)
zero_cols = [ col for col, is_zero in ((X == 0).sum() == X.shape[0]).items() if is_zero ]

X.drop(zero_cols, axis=1, inplace=True)
X.drop(columns = ['PID'], inplace = True)

## 3.4 Manufacture Variables

In [9]:
seasons = {'Spr':[3,4,5], 'Sum':[6,7,8], 'Fal':[9,10,11], 'Win':[12,1,2]}
X['Season'] = X['Mo Sold'].map(lambda x: [i for i in seasons if x in seasons[i]][0])
X = pd.get_dummies(X, columns=['Season'], drop_first = True)
X.drop(columns = ['Mo Sold'], inplace = True)
X['Bathroom'] = X['Full Bath'] + X['Bsmt Full Bath'] + 0.5 * X['Half Bath'] + 0.5 * X['Bsmt Half Bath']
X.drop(columns = ['Full Bath', 'Bsmt Full Bath', 'Half Bath', 'Bsmt Half Bath'], inplace = True)
X.drop(columns = ['Mas Vnr Type_None', 'Misc Feature_None'], inplace = True)

In [10]:
# Drop variables with skew data from the pairplot above.
X.drop(columns = ['MS Zoning_I (all)', 'Foundation_Wood', 'Condition 1_RRNn',
                  'Roof Style_Shed', 'Roof Matl_WdShngl', 'Roof Matl_WdShake', 
                  'Roof Style_Shed', 'Exterior 1st_AsbShng', 'Exterior 1st_BrkComm', 
                  'Exterior 2nd_AsphShn', 'Exterior 2nd_CBlock', 'Misc Feature_Othr',
                  'Mas Vnr Type_CBlock', 'Sale Type_VWD', 'Heating_Floor',
                  'Roof Matl_Roll', 'Roof Matl_Metal', 'Exterior 1st_PreCast',
                  'Exterior 2nd_PreCast', 'Exterior 2nd_Other'], inplace = True)

# 4.0 Predicting SalePrice

In [11]:
# Export data for use in next dataset
X.to_csv("datasets/test_v2_Kaggle.csv", index = False)

In [12]:
X = pd.read_csv('datasets/train_v3_preprocessed.csv')
y = X.pop('SalePrice')
y = np.log(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rs = RobustScaler()
X_train_rs = rs.fit_transform(X_train)
X_test_rs = rs.transform(X_test)
r_alphas = np.logspace(0, 10, 100)
ridge_cv = RidgeCV(alphas=r_alphas, scoring='r2', cv=5).fit(X_train_rs, y_train)
print(ridge_cv.score(X_train_rs, y_train))
print(ridge_cv.score(X_test_rs, y_test))

ridge_cv.alpha_

0.9165512192717161
0.8843649131827771


12.91549665014884

In [13]:
rg = Ridge(alpha = 13.84)
rs = RobustScaler()
X_train_rs = rs.fit_transform(X_train)
X_test_rs = rs.transform(X_test)
rg.fit(X_train_rs, y_train)

Ridge(alpha=13.84)

In [14]:
X = pd.read_csv('datasets/test_v2_Kaggle.csv')
y = X.pop('Id')
X_rs = rs.fit_transform(X)
X['SalePrice'] = np.exp(rg.predict(X_rs))
X['Id'] = y
submission = X[['Id','SalePrice']]
submission.to_csv("datasets/test_v3_Submit.csv", index = False)

Kaggle Submission Score: 28705